In [1]:
!python spark.py symbol_list.csv

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6386a3b6-79e9-4de2-9c7c-5f9e24af49e8;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.2.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.2.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import sys


# replace host cassandra !!! 
spark = SparkSession.builder\
            .config("spark.app.name", "StockDataAnalyzer")\
            .config("spark.master", "spark://spark-master:7077")\
            .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.2.0")\
            .config("spark.cassandra.connection.host", "cassandra")\
            .config("spark.cassandra.auth.username", "cassandra")\
            .config("spark.cassandra.auth.password", "cassandra")\
            .enableHiveSupport()\
            .getOrCreate()
spark.sparkContext.setLogLevel("WARN")

df = spark.read.format('csv')\
    .option('header', True)\
    .option('inferSchema', True)\
    .load("hdfs://namenode:9000/stockData/2024/12/30/stockData.1735537315")
print(df.head(10))
df.show()
if spark.sparkContext._jsc.sc().isStopped():
    print("The SparkContext is stopped.")
else:
    print("The SparkContext is active.")

df = df.select(
    col('Symbol').alias('symbol'),
    col(' Trading date').alias('trading_date'),
    col(' High').alias('high'), col(' Low').alias('low'), col(' Open').alias('open'), col(' Close').alias('close'), col(' Volume').alias('volume')
)

# save data into cassandra
df.write.format('org.apache.spark.sql.cassandra')\
        .mode('append')\
        .options(table='stock_data', keyspace='stock')\
        .save()
    

[Row(Symbol='HSV',  Open=4000.0,  High=4200.0,  Low=3900.0,  Close=4011.0,  Volume=104747,  Trading date=datetime.datetime(2024, 12, 26, 0, 0)), Row(Symbol='CST',  Open=24100.0,  High=25600.0,  Low=24100.0,  Close=25103.0,  Volume=123844,  Trading date=datetime.datetime(2024, 12, 26, 0, 0)), Row(Symbol='BVL',  Open=9600.0,  High=9600.0,  Low=9400.0,  Close=9464.0,  Volume=2500,  Trading date=datetime.datetime(2024, 12, 26, 0, 0)), Row(Symbol='SGI',  Open=14300.0,  High=14300.0,  Low=14300.0,  Close=14300.0,  Volume=100,  Trading date=datetime.datetime(2024, 12, 26, 0, 0)), Row(Symbol='TOS',  Open=75000.0,  High=75000.0,  Low=74000.0,  Close=74189.0,  Volume=10000,  Trading date=datetime.datetime(2024, 12, 26, 0, 0)), Row(Symbol='VTZ',  Open=17500.0,  High=17600.0,  Low=17300.0,  Close=17400.0,  Volume=1038806,  Trading date=datetime.datetime(2024, 12, 26, 0, 0)), Row(Symbol='SSH',  Open=67300.0,  High=67300.0,  Low=66800.0,  Close=67033.0,  Volume=50600,  Trading date=datetime.datetime

24/12/30 05:46:46 WARN PlainTextAuthProviderBase: [] cassandra/172.18.0.10:9042 did not send an authentication challenge; This is suspicious because the driver expects authentication
24/12/30 05:46:46 WARN PlainTextAuthProviderBase: [] cassandra/172.18.0.10:9042 did not send an authentication challenge; This is suspicious because the driver expects authentication


AnalysisException: Couldn't find stock or any similarly named keyspaces

In [6]:
# save data into cassandra
df.write.format('org.apache.spark.sql.cassandra')\
        .mode('append')\
        .options(table='stock_data', keyspace='stock')\
        .save()



Py4JJavaError: An error occurred while calling o113.save.
: java.lang.ClassNotFoundException: Failed to find data source: org.apache.spark.sql.cassandra. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:692)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:746)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:993)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:311)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:301)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.inv

In [ ]:
if spark.sparkContext._jsc.sc().isStopped():
    print("The SparkContext is stopped.")
else:
    print("The SparkContext is active.")

In [3]:
import pyspark
print(pyspark.__version__)

3.1.2


In [4]:
# Create a DataFrame
data = [("Alice", 1), ("Bob", 2), ("Cathy", 3)]
df = spark.createDataFrame(data, ["Name", "Id"])

# Show the DataFrame
df.show()


24/12/29 12:35:30 WARN TaskSetManager: Lost task 0.0 in stage 3.0 (TID 3) (172.18.0.13 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/spark/python/lib/pyspark.zip/pyspark/worker.py", line 477, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 3.7 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:652)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:635)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:470)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collectio

Py4JJavaError: An error occurred while calling o56.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 4 times, most recent failure: Lost task 0.3 in stage 3.0 (TID 6) (172.18.0.13 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/spark/python/lib/pyspark.zip/pyspark/worker.py", line 477, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 3.7 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:652)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:635)
	at org.apache.spark.api.python.BasePythonRunner